# G-Tube Project

In [20]:
import pandas as pd
import numpy as np
import glob
import re
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# To-do

# 1 hour before : ('first pause')
# 1 hour after : ('second pause')
# Notes - induction -> closing for During. Variable amout of time
# create new variable: cStO2, rStO2

## Background

Premature infants undergo G-Tube Surgery. Data was collected for around 24 hours before and 24 hours after (not exact). This data was collected using NIRS which interfaced with the Moberg Machine. The first NIRS probe is placed on the infant's brain and the second probe is placed in the flank for renal StO2. 

EEG leads done by nurses. Pause monitoring for transport period. Stopped right before transport, restarted as soon as baby gets put onto the table.
Pauses (max 20 mins).

For this project, since there is no specific hypothesis, data will be visualized and exploratory data analysis will be used to better inform further analysis.

## Data Preparation and Cleaning

Data was downloaded from a private Google Drive server and file names were de-identified with this naming scheme: GTP001, GTP002, etc. Each file is concatenated into one dataframe for easier analysis.
Only relevant data was filtered out from the raw dataset to form the analysis dataframe. FTOE was calcualted from SpO2 and StO2.

## Data Features

|Feature Name   | Description   | Variable Type   |  
|---|---|---|
|Subject   | ID of Subject  | label  | 
|HR   | Heart Rate  | quantitative  | 
|RR   | Respiratory Rate  | quantitative  | 
|SpO2   | Peripheral Oxygen Saturation   | quantitative  | 
|MAP   | Mean Arterial Pressure  | quantitative  | 
|StO2_1   | Local tissue oxygenation - brain  | quantitative  | 
|StO2_2   | Local tissue oxygenation - renal  | quantitative  | 
|FTOE_1   | Fractional tissue oxygenation extraction - brain  | quantitative  | 
|FTOE_2   | Fractional tissue oxygenation extraction - renal  | quantitative  | 
|Annotation   | Patient notes  | label  | 

In [10]:
files_lst= glob.glob(r'C:\Users\jctan\Documents\G-Tube Project\raws\*.csv', recursive=True) # Output 1

In [11]:
df_lst = []


In [12]:
for f in files_lst:
    # load file
    df = pd.read_csv(f, sep=',', low_memory=False)
    
    # Assign ID and Minutes
    df['Subject'] = re.search(r'GTP(.*?).csv', f).group(1).lstrip("0")
    df['Minutes'] = np.arange(1,len(df)+1)
    
    # EEG
    df['C3-P3'] = df['ADR,C3-P3,Numeric,Float,Amp1020,ADR_DL=0.5,ADR_DH=3.5,ADR_AL=8,ADR_AH=12.5:Mean']
    df['C4-P4'] = df['ADR,C4-P4,Numeric,Float,Amp1020,ADR_DL=0.5,ADR_DH=3.5,ADR_AL=8,ADR_AH=12.5:Mean']

    # Vitals
    df['HR'] = df['HR,na,Numeric,Float,IntelliVue (bpm):Mean']
    df['RR'] = df['RR,na,Numeric,Float,IntelliVue (BPM):Mean']
    df['SpO2'] = df['SpO2,na,Numeric,Float,IntelliVue (%):Mean']
    df['MAP'] = df['NBP,Mean,Numeric,Float,IntelliVue (mmHg):Mean']

    # NIRS
    df['StO2_1'] = df[df.columns[df.columns.str.startswith('StO2,1') & df.columns.str.endswith('Mean')]]
    df['StO2_2'] = df[df.columns[df.columns.str.startswith('StO2,2') & df.columns.str.endswith('Mean')]]
    
    df['FTOE_1'] = (df['SpO2'] - df['StO2_1']) / df['SpO2'] * 100 # (SpO2 - StO2) / SpO2
    df['FTOE_2'] = (df['SpO2'] - df['StO2_2']) / df['SpO2'] * 100
    
    df['Annotation'] = df.filter(regex='Notes')
    df_lst.append(df)

In [15]:
dff = pd.concat(df_lst, ignore_index=True)

C:\Users\jctan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


## Data Subsetting

For now, remove all EEG data. Focus on one hour around time = 0 which for now will be "prepping"

In [16]:
varlst = ['Subject', 'HR', 'RR', 'SpO2', 'MAP', 'StO2_1', 'StO2_2', 'FTOE_1', 'FTOE_2', 'Annotation'] # took out EEG for now

data = dff[varlst] 

In [17]:
dct_temp = data.loc[data['Annotation'] == 'prepping']['Subject'].to_dict() # For each subject, find where 'prepping' occurs and set it as a dict value.
index_dct = dict((v,k) for k,v in dct_temp.items()) # Have to switch key and value in the temporary dict

In [18]:
# Split up the data into 60 mins before and 60 mins after procedure

before_lst = []
after_lst = []

Subject_lst = ['1', '2', '3', '4', '5', '6', '7']

def get_relevant_times(data):
    for i in Subject_lst:
        before = data[index_dct[i] - 60 : index_dct[i]]
        after = data[index_dct[i] : index_dct[i] + 60]
        before_lst.append(before)
        after_lst.append(after)
    
    df_before = pd.concat(before_lst)
    df_after = pd.concat(after_lst)
    return df_before, df_after

In [65]:
dfb, dfa = get_relevant_times(data)

In [66]:
# Function to display tables side-by-side
# Not working in JupyterLab?

from IPython.display import display_html
def display_sbs(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [89]:
print('Summary Statistics')

for i in varlst[1:]:
    dfb_summ = dfb[['Subject', i]].groupby('Subject').describe()
    dfa_summ = dfa[['Subject', i]].groupby('Subject').describe()
    display_sbs(dfb_summ, dfa_summ)
    
# dfb.boxplot(by='Subject', column='HR')
# plt.title('HR' + ' 1 Hr Before Procedure')


Summary Statistics


HR 
 
 
 
 count 
 mean 
 std 
 min 
 25% 
 50% 
 75% 
 max 
 
 
 Subject 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 62.0 
 162.097516 
 26.255357 
 100.200 
 137.69100 
 168.5760 
 185.09925 
 193.085 
 
 
 2 
 104.0 
 167.193635 
 16.969053 
 125.103 
 158.82075 
 166.7585 
 180.04825 
 192.051 
 
 
 3 
 120.0 
 140.007967 
 16.004026 
 110.966 
 127.42800 
 140.7490 
 153.77550 
 173.143 
 
 
 4 
 68.0 
 134.027471 
 12.688804 
 114.119 
 125.56900 
 130.4535 
 139.43100 
 169.017 
 
 
 5 
 102.0 
 138.653333 
 16.365312 
 118.356 
 124.96125 
 135.8310 
 147.63700 
 192.831 
 
 
 6 
 64.0 
 160.341781 
 15.536154 
 133.271 
 150.66650 
 162.0090 
 172.61875 
 185.250 
 
 
 7 
 90.0 
 168.395400 
 6.275163 
 157.542 
 163.93200 
 167.7970 
 172.96600 
 184.729 
 
 
 
 
 
 
 HR 
 
 
 
 count 
 mean 
 std 
 min 
 25% 
 50% 
 75% 
 max 
 
 
 Subject 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 0.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 2 
 24.0 
 133.699917 
 20.613067 
 109.864 
 118.06275 
 130.1855 
 139.02125 
 176.305 
 
 
 3 
 120.0 
 129.287750 
 12.062717 
 107.508 
 121.93225 
 126.2795 
 137.94250 
 158.610 
 
 
 4 
 14.0 
 156.443571 
 4.315376 
 150.431 
 154.38100 
 155.5170 
 158.79175 
 164.500 
 
 
 5 
 6.0 
 136.295333 
 2.184887 
 133.695 
 134.43350 
 136.6490 
 138.06875 
 138.542 
 
 
 6 
 0.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 7 
 60.0 
 148.226433 
 14.527593 
 111.169 
 143.56900 
 149.0400 
 156.93100 
 170.542

RR 
 
 
 
 count 
 mean 
 std 
 min 
 25% 
 50% 
 75% 
 max 
 
 
 Subject 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 62.0 
 48.957397 
 9.973045 
 3.775860e+01 
 41.631375 
 45.05170 
 53.919100 
 70.6207 
 
 
 2 
 104.0 
 46.977079 
 7.299158 
 3.272340e+01 
 41.077550 
 46.11020 
 51.627150 
 61.7119 
 
 
 3 
 120.0 
 41.268488 
 6.350994 
 3.077590e+01 
 36.516975 
 40.40080 
 45.043725 
 55.6897 
 
 
 4 
 70.0 
 34.118274 
 6.592533 
 4.390000e-42 
 33.467575 
 35.03390 
 36.678000 
 41.6379 
 
 
 5 
 102.0 
 37.495675 
 7.365479 
 2.642370e+01 
 30.741550 
 37.84480 
 41.352525 
 59.4746 
 
 
 6 
 64.0 
 43.439334 
 10.883246 
 2.162070e+01 
 35.694900 
 41.90675 
 48.742050 
 70.1525 
 
 
 7 
 90.0 
 57.026784 
 8.649702 
 4.300000e+01 
 50.465500 
 55.79660 
 62.322000 
 75.7288 
 
 
 
 
 
 
 RR 
 
 
 
 count 
 mean 
 std 
 min 
 25% 
 50% 
 75% 
 max 
 
 
 Subject 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 0.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 2 
 24.0 
 38.082458 
 5.924761 
 27.6610 
 34.375600 
 40.45630 
 42.750325 
 44.9322 
 
 
 3 
 114.0 
 36.986686 
 8.777950 
 24.3051 
 29.793100 
 36.28810 
 43.728800 
 61.9492 
 
 
 4 
 14.0 
 49.136086 
 8.740375 
 30.5000 
 47.435925 
 50.52540 
 55.770025 
 57.1724 
 
 
 5 
 6.0 
 42.439700 
 5.160390 
 35.7797 
 38.241550 
 45.62710 
 45.841000 
 45.9123 
 
 
 6 
 0.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 7 
 60.0 
 38.806463 
 13.453281 
 27.1356 
 29.915300 
 33.76475 
 39.965500 
 76.0000

SpO2 
 
 
 
 count 
 mean 
 std 
 min 
 25% 
 50% 
 75% 
 max 
 
 
 Subject 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 120.0 
 96.062472 
 6.478114 
 68.4310 
 94.788175 
 99.63240 
 100.000000 
 100.0 
 
 
 2 
 120.0 
 99.594788 
 0.869679 
 93.8121 
 99.556550 
 99.87790 
 100.000000 
 100.0 
 
 
 3 
 120.0 
 99.462585 
 0.813244 
 95.9509 
 99.433100 
 99.78035 
 99.963975 
 100.0 
 
 
 4 
 118.0 
 98.992505 
 1.947391 
 88.4746 
 99.189700 
 99.48280 
 99.796600 
 100.0 
 
 
 5 
 120.0 
 99.401257 
 2.457657 
 81.1017 
 99.637900 
 99.91455 
 100.000000 
 100.0 
 
 
 6 
 86.0 
 98.642002 
 2.848621 
 80.7500 
 98.757625 
 98.94830 
 99.437925 
 100.0 
 
 
 7 
 112.0 
 97.990375 
 4.422450 
 76.9500 
 98.728475 
 100.00000 
 100.000000 
 100.0 
 
 
 
 
 
 
 SpO2 
 
 
 
 count 
 mean 
 std 
 min 
 25% 
 50% 
 75% 
 max 
 
 
 Subject 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 120.0 
 99.587923 
 0.983765 
 94.3469 
 99.584725 
 100.00000 
 100.0000 
 100.0000 
 
 
 2 
 120.0 
 98.245125 
 3.704259 
 82.7085 
 99.841650 
 99.98810 
 100.0000 
 100.0000 
 
 
 3 
 120.0 
 99.486655 
 0.844755 
 94.4810 
 99.268875 
 99.77410 
 99.9944 
 100.0000 
 
 
 4 
 120.0 
 98.139490 
 1.697906 
 86.1200 
 98.000000 
 98.40675 
 98.9153 
 99.3621 
 
 
 5 
 120.0 
 99.239668 
 2.321639 
 88.1552 
 100.000000 
 100.00000 
 100.0000 
 100.0000 
 
 
 6 
 84.0 
 99.972719 
 0.175732 
 98.8542 
 100.000000 
 100.00000 
 100.0000 
 100.0000 
 
 
 7 
 120.0 
 96.623640 
 7.953820 
 53.7500 
 97.982750 
 99.69245 
 100.0000 
 100.0000

MAP 
 
 
 
 count 
 mean 
 std 
 min 
 25% 
 50% 
 75% 
 max 
 
 
 Subject 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 34.0 
 49.352941 
 18.217345 
 29.0 
 31.0 
 45.0 
 70.0 
 70.0000 
 
 
 2 
 98.0 
 63.906496 
 5.167792 
 45.0 
 66.0 
 66.0 
 66.0 
 66.8511 
 
 
 3 
 10.0 
 46.200000 
 12.354037 
 35.0 
 37.0 
 38.0 
 60.0 
 61.0000 
 
 
 4 
 4.0 
 56.000000 
 0.000000 
 56.0 
 56.0 
 56.0 
 56.0 
 56.0000 
 
 
 5 
 90.0 
 62.000000 
 0.000000 
 62.0 
 62.0 
 62.0 
 62.0 
 62.0000 
 
 
 6 
 2.0 
 67.000000 
 0.000000 
 67.0 
 67.0 
 67.0 
 67.0 
 67.0000 
 
 
 7 
 90.0 
 57.117222 
 11.622631 
 43.0 
 43.0 
 67.0 
 67.0 
 67.0000 
 
 
 
 
 
 
 MAP 
 
 
 
 count 
 mean 
 std 
 min 
 25% 
 50% 
 75% 
 max 
 
 
 Subject 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 42.0 
 38.000000 
 6.104776 
 31.0 
 34.0 
 36.0 
 40.00000 
 51.0 
 
 
 2 
 54.0 
 56.388526 
 14.709580 
 40.0 
 43.0 
 48.0 
 74.37255 
 75.0 
 
 
 3 
 40.0 
 36.400000 
 9.388482 
 27.0 
 31.0 
 35.0 
 37.25000 
 73.0 
 
 
 4 
 12.0 
 60.000000 
 0.000000 
 60.0 
 60.0 
 60.0 
 60.00000 
 60.0 
 
 
 5 
 4.0 
 44.000000 
 0.000000 
 44.0 
 44.0 
 44.0 
 44.00000 
 44.0 
 
 
 6 
 0.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 7 
 58.0 
 79.034483 
 2.286233 
 78.0 
 78.0 
 78.0 
 78.00000 
 84.0

StO2_1 
 
 
 
 count 
 mean 
 std 
 min 
 25% 
 50% 
 75% 
 max 
 
 
 Subject 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 120.0 
 73.203413 
 4.851744 
 65.7200 
 70.090000 
 72.03705 
 75.841550 
 85.6154 
 
 
 2 
 120.0 
 80.861588 
 3.555182 
 76.2000 
 78.682675 
 80.15110 
 81.094700 
 90.2222 
 
 
 3 
 120.0 
 86.684375 
 1.189209 
 83.1200 
 85.826950 
 86.86540 
 87.680350 
 88.5000 
 
 
 4 
 118.0 
 79.081505 
 3.843667 
 73.8800 
 76.211525 
 78.52000 
 80.823450 
 90.8125 
 
 
 5 
 66.0 
 80.891315 
 5.229498 
 73.2800 
 75.846200 
 81.03700 
 85.571400 
 91.0000 
 
 
 6 
 120.0 
 68.584808 
 3.241179 
 61.0000 
 66.203000 
 69.25460 
 70.442325 
 77.1538 
 
 
 7 
 120.0 
 79.072892 
 4.577703 
 67.6667 
 77.230800 
 78.01925 
 78.847625 
 89.0000 
 
 
 
 
 
 
 StO2_1 
 
 
 
 count 
 mean 
 std 
 min 
 25% 
 50% 
 75% 
 max 
 
 
 Subject 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 120.0 
 74.359255 
 4.228128 
 67.0000 
 70.942300 
 75.00000 
 76.555550 
 83.6000 
 
 
 2 
 120.0 
 81.464865 
 6.312369 
 71.2400 
 76.875000 
 79.71155 
 88.576900 
 89.0000 
 
 
 3 
 120.0 
 85.265507 
 3.793424 
 79.8000 
 82.300775 
 84.03615 
 87.190000 
 96.4000 
 
 
 4 
 120.0 
 73.449567 
 5.580388 
 63.3462 
 69.778875 
 73.51925 
 77.537025 
 85.3571 
 
 
 5 
 120.0 
 77.408117 
 5.847483 
 67.0769 
 73.778850 
 76.77565 
 79.258925 
 88.0370 
 
 
 6 
 120.0 
 74.199217 
 8.618087 
 64.6923 
 67.000000 
 71.12965 
 82.511550 
 89.0000 
 
 
 7 
 120.0 
 69.565298 
 6.205247 
 59.4074 
 66.457225 
 68.13390 
 72.123575 
 84.8889

StO2_2 
 
 
 
 count 
 mean 
 std 
 min 
 25% 
 50% 
 75% 
 max 
 
 
 Subject 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 120.0 
 80.921543 
 10.573076 
 5.940000e+01 
 73.057675 
 81.32980 
 91.493450 
 93.0000 
 
 
 2 
 120.0 
 78.642805 
 6.719130 
 6.992310e+01 
 72.736900 
 75.34615 
 84.869700 
 90.9615 
 
 
 3 
 120.0 
 79.878790 
 4.585914 
 7.364000e+01 
 75.905175 
 78.92075 
 83.750000 
 91.0769 
 
 
 4 
 120.0 
 83.828907 
 11.738521 
 4.390000e-42 
 82.460000 
 85.19230 
 87.846550 
 92.0000 
 
 
 5 
 120.0 
 87.086622 
 5.207539 
 7.254170e+01 
 84.422300 
 89.03175 
 91.287075 
 92.0000 
 
 
 6 
 120.0 
 71.876868 
 7.064140 
 6.180000e+01 
 66.298750 
 69.25000 
 76.650775 
 87.0000 
 
 
 7 
 120.0 
 85.157527 
 4.247956 
 7.744440e+01 
 82.478975 
 84.10000 
 88.113225 
 92.4000 
 
 
 
 
 
 
 StO2_2 
 
 
 
 count 
 mean 
 std 
 min 
 25% 
 50% 
 75% 
 max 
 
 
 Subject 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 120.0 
 85.424565 
 5.145442 
 77.4074 
 80.068300 
 85.26495 
 91.798100 
 92.5769 
 
 
 2 
 110.0 
 86.033264 
 6.487046 
 69.8077 
 81.855750 
 87.00000 
 91.000000 
 97.3333 
 
 
 3 
 120.0 
 81.113637 
 4.248147 
 71.5600 
 77.813850 
 80.15925 
 83.836550 
 91.8077 
 
 
 4 
 120.0 
 88.206750 
 4.113662 
 80.3462 
 84.817300 
 89.07690 
 92.000000 
 93.0000 
 
 
 5 
 120.0 
 90.873523 
 4.541643 
 69.8846 
 92.000000 
 92.00000 
 92.008925 
 93.0000 
 
 
 6 
 120.0 
 71.061727 
 6.789173 
 61.8519 
 65.020650 
 68.88460 
 76.833350 
 84.9630 
 
 
 7 
 120.0 
 80.375185 
 7.425620 
 56.9259 
 77.538475 
 80.74040 
 84.916675 
 91.0000

FTOE_1 
 
 
 
 count 
 mean 
 std 
 min 
 25% 
 50% 
 75% 
 max 
 
 
 Subject 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 120.0 
 23.555442 
 5.832385 
 -0.775233 
 19.797021 
 25.436573 
 28.045311 
 30.730800 
 
 
 2 
 120.0 
 18.806066 
 3.566619 
 9.617304 
 18.787481 
 19.609318 
 21.195881 
 23.461908 
 
 
 3 
 120.0 
 12.844299 
 1.200225 
 9.813972 
 12.037200 
 12.699306 
 13.650975 
 15.417094 
 
 
 4 
 118.0 
 20.102225 
 3.769763 
 9.187500 
 18.283866 
 20.661617 
 22.728597 
 25.937269 
 
 
 5 
 66.0 
 18.221563 
 5.586849 
 6.337968 
 13.891149 
 17.197469 
 24.153800 
 26.294483 
 
 
 6 
 86.0 
 30.131708 
 3.435598 
 12.074303 
 29.324244 
 29.723706 
 31.773470 
 34.613747 
 
 
 7 
 112.0 
 19.617812 
 5.574262 
 -1.676413 
 16.172296 
 21.595723 
 22.284875 
 32.333300 
 
 
 
 
 
 
 FTOE_1 
 
 
 
 count 
 mean 
 std 
 min 
 25% 
 50% 
 75% 
 max 
 
 
 Subject 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 120.0 
 25.314732 
 4.510387 
 14.952161 
 23.365862 
 25.000000 
 28.980258 
 33.000000 
 
 
 2 
 120.0 
 17.102996 
 5.232179 
 6.950313 
 11.372766 
 17.409850 
 22.182375 
 23.980311 
 
 
 3 
 120.0 
 14.292869 
 3.782301 
 3.177669 
 12.140364 
 15.500000 
 17.390511 
 18.835010 
 
 
 4 
 120.0 
 25.138024 
 5.781791 
 12.172759 
 20.977871 
 24.737949 
 29.005313 
 35.722874 
 
 
 5 
 120.0 
 21.968673 
 5.986324 
 6.983725 
 18.856261 
 22.586900 
 25.951900 
 32.923100 
 
 
 6 
 84.0 
 30.988173 
 3.409348 
 24.000000 
 28.177053 
 32.057700 
 34.000000 
 35.307700 
 
 
 7 
 120.0 
 27.580554 
 7.939025 
 -10.525395 
 24.444457 
 29.673100 
 32.068799 
 39.000000

FTOE_2 
 
 
 
 count 
 mean 
 std 
 min 
 25% 
 50% 
 75% 
 max 
 
 
 Subject 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 120.0 
 15.725145 
 9.695034 
 -7.519838 
 7.430445 
 14.735745 
 21.127674 
 36.773660 
 
 
 2 
 120.0 
 21.044776 
 6.608746 
 9.035407 
 15.019253 
 24.070289 
 26.556290 
 30.069767 
 
 
 3 
 120.0 
 19.703525 
 4.307361 
 8.864956 
 16.250000 
 20.950272 
 22.876515 
 25.089454 
 
 
 4 
 118.0 
 13.863404 
 4.321596 
 5.249999 
 10.311701 
 13.657668 
 16.224218 
 24.510469 
 
 
 5 
 120.0 
 12.351960 
 5.386964 
 -3.289080 
 8.615050 
 10.519162 
 14.887917 
 27.119039 
 
 
 6 
 86.0 
 26.118370 
 7.790035 
 12.406113 
 21.694154 
 27.046956 
 33.252198 
 37.275058 
 
 
 7 
 112.0 
 13.296199 
 5.071658 
 -9.005848 
 10.535541 
 14.394591 
 16.339350 
 22.555600 
 
 
 
 
 
 
 FTOE_2 
 
 
 
 count 
 mean 
 std 
 min 
 25% 
 50% 
 75% 
 max 
 
 
 Subject 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 120.0 
 14.223680 
 5.062031 
 7.423100 
 8.000000 
 14.291862 
 19.829925 
 22.249956 
 
 
 2 
 110.0 
 12.599587 
 7.143112 
 -17.682342 
 9.000000 
 13.000000 
 17.572133 
 23.567263 
 
 
 3 
 120.0 
 18.478691 
 4.000756 
 8.192300 
 15.538586 
 19.515109 
 21.833808 
 24.259904 
 
 
 4 
 120.0 
 10.101992 
 4.308283 
 -1.557710 
 7.070707 
 8.870239 
 13.189839 
 18.301527 
 
 
 5 
 120.0 
 8.425860 
 4.141471 
 1.698710 
 7.000000 
 8.000000 
 8.000000 
 30.115400 
 
 
 6 
 84.0 
 32.160864 
 5.043369 
 19.541700 
 31.000000 
 34.019250 
 35.333300 
 38.148100 
 
 
 7 
 120.0 
 16.617655 
 6.242361 
 -5.908651 
 14.178027 
 17.364771 
 20.786991 
 28.128382

Annotation 
 
 
 
 count 
 unique 
 top 
 freq 
 
 
 Subject 
 
 
 
 
 
 
 
 
 1 
 16 
 8 
 stomach insufflation 
 2 
 
 
 2 
 8 
 4 
 1st in OR 
 2 
 
 
 3 
 12 
 5 
 induction 
 4 
 
 
 4 
 6 
 3 
 induction meds 
 2 
 
 
 5 
 8 
 4 
 positioning on table style="display:inline" 
 2 
 
 
 6 
 50 
 4 
 IV attempts 
 44 
 
 
 7 
 4 
 2 
 positioning 
 2 
 
 
 
 
 
 
 Annotation 
 
 
 
 count 
 unique 
 top 
 freq 
 
 
 Subject 
 
 
 
 
 
 
 
 
 1 
 64 
 26 
 suturing umbilicus 
 10 
 
 
 2 
 40 
 19 
 dilation 
 4 
 
 
 3 
 42 
 18 
 stomach insufflated 
 4 
 
 
 4 
 52 
 22 
 suturing 
 6 
 
 
 5 
 38 
 18 
 gtube suture 
 4 
 
 
 6 
 76 
 23 
 iv attempts 
 30 
 
 
 7 
 40 
 20 
 prepping 
 2

## Correlation

In [91]:
display_sbs(dfb.groupby('Subject').corr(), dfa.groupby('Subject').corr())

FTOE_1 
 FTOE_2 
 HR 
 MAP 
 RR 
 SpO2 
 StO2_1 
 StO2_2 
 
 
 Subject 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 FTOE_1 
 1.000000 
 0.268242 
 -0.143006 
 0.106897 
 0.147070 
 0.616842 
 -0.415282 
 0.127900 
 
 
 FTOE_2 
 0.268242 
 1.000000 
 0.707342 
 0.816585 
 -0.089225 
 0.056141 
 -0.192582 
 -0.828158 
 
 
 HR 
 -0.143006 
 0.707342 
 1.000000 
 NaN 
 -0.176452 
 -0.387881 
 -0.604053 
 -0.843836 
 
 
 MAP 
 0.106897 
 0.816585 
 NaN 
 1.000000 
 NaN 
 -0.054016 
 -0.126266 
 -0.782168 
 
 
 RR 
 0.147070 
 -0.089225 
 -0.176452 
 NaN 
 1.000000 
 0.223707 
 0.211865 
 0.198208 
 
 
 SpO2 
 0.616842 
 0.056141 
 -0.387881 
 -0.054016 
 0.223707 
 1.000000 
 0.453344 
 0.508798 
 
 
 StO2_1 
 -0.415282 
 -0.192582 
 -0.604053 
 -0.126266 
 0.211865 
 0.453344 
 1.000000 
 0.407757 
 
 
 StO2_2 
 0.127900 
 -0.828158 
 -0.843836 
 -0.782168 
 0.198208 
 0.508798 
 0.407757 
 1.000000 
 
 
 2 
 FTOE_1 
 1.000000 
 0.714180 
 -0.060458 
 0.658417 
 0.371810 
 0.108539 
 -0.978610 
 -0.690616 
 
 
 FTOE_2 
 0.714180 
 1.000000 
 0.756079 
 0.384947 
 0.116950 
 -0.131886 
 -0.741918 
 -0.995073 
 
 
 HR 
 -0.060458 
 0.756079 
 1.000000 
 -0.101496 
 -0.131096 
 -0.231594 
 -0.038473 
 -0.775723 
 
 
 MAP 
 0.658417 
 0.384947 
 -0.101496 
 1.000000 
 0.029756 
 -0.157456 
 -0.716072 
 -0.395209 
 
 
 RR 
 0.371810 
 0.116950 
 -0.131096 
 0.029756 
 1.000000 
 0.251953 
 -0.309894 
 -0.081858 
 
 
 SpO2 
 0.108539 
 -0.131886 
 -0.231594 
 -0.157456 
 0.251953 
 1.000000 
 0.098210 
 0.229464 
 
 
 StO2_1 
 -0.978610 
 -0.741918 
 -0.038473 
 -0.716072 
 -0.309894 
 0.098210 
 1.000000 
 0.738531 
 
 
 StO2_2 
 -0.690616 
 -0.995073 
 -0.775723 
 -0.395209 
 -0.081858 
 0.229464 
 0.738531 
 1.000000 
 
 
 3 
 FTOE_1 
 1.000000 
 -0.141332 
 -0.441140 
 -0.497538 
 0.198734 
 0.303593 
 -0.822235 
 0.174678 
 
 
 FTOE_2 
 -0.141332 
 1.000000 
 0.319268 
 0.485571 
 0.003514 
 -0.398438 
 -0.096230 
 -0.992058 
 
 
 HR 
 -0.441140 
 0.319268 
 1.000000 
 0.865054 
 -0.426389 
 -0.466241 
 0.164640 
 -0.363822 
 
 
 MAP 
 -0.497538 
 0.485571 
 0.865054 
 1.000000 
 -0.381390 
 -0.956478 
 -0.116143 
 -0.639770 
 
 
 RR 
 0.198734 
 0.003514 
 -0.426389 
 -0.381390 
 1.000000 
 0.165576 
 -0.101572 
 0.019796 
 
 
 SpO2 
 0.303593 
 -0.398438 
 -0.466241 
 -0.956478 
 0.165576 
 1.000000 
 0.292611 
 0.510584 
 
 
 StO2_1 
 -0.822235 
 -0.096230 
 0.164640 
 -0.116143 
 -0.101572 
 0.292611 
 1.000000 
 0.129744 
 
 
 StO2_2 
 0.174678 
 -0.992058 
 -0.363822 
 -0.639770 
 0.019796 
 0.510584 
 0.129744 
 1.000000 
 
 
 4 
 FTOE_1 
 1.000000 
 -0.261606 
 -0.640216 
 NaN 
 -0.269865 
 0.155292 
 -0.908274 
 0.330486 
 
 
 FTOE_2 
 -0.261606 
 1.000000 
 0.584996 
 NaN 
 0.319699 
 0.228152 
 0.351341 
 -0.917044 
 
 
 HR 
 -0.640216 
 0.584996 
 1.000000 
 NaN 
 0.262779 
 -0.101672 
 0.622449 
 -0.592152 
 
 
 MAP 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 RR 
 -0.269865 
 0.319699 
 0.262779 
 NaN 
 1.000000 
 0.377063 
 0.291383 
 0.833410 
 
 
 SpO2 
 0.155292 
 0.228152 
 -0.101672 
 NaN 
 0.377063 
 1.000000 
 0.271979 
 0.178772 
 
 
 StO2_1 
 -0.908274 
 0.351341 
 0.622449 
 NaN 
 0.291383 
 0.271979 
 1.000000 
 -0.246120 
 
 
 StO2_2 
 0.330486 
 -0.917044 
 -0.592152 
 NaN 
 0.833410 
 0.178772 
 -0.246120 
 1.000000 
 
 
 5 
 FTOE_1 
 1.000000 
 0.040565 
 0.123491 
 NaN 
 0.355719 
 0.378589 
 -0.843294 
 0.176806 
 
 
 FTOE_2 
 0.040565 
 1.000000 
 0.709782 
 NaN 
 0.181909 
 0.279010 
 0.206943 
 -0.887300 
 
 
 HR 
 0.123491 
 0.709782 
 1.000000 
 NaN 
 0.532644 
 -0.568753 
 -0.176011 
 -0.726473 
 
 
 MAP 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 RR 
 0.355719 
 0.181909 
 0.532644 
 NaN 
 1.000000 
 -0.369283 
 -0.379369 
 -0.203855 
 
 
 SpO2 
 0.378589 
 0.279010 
 -0.568753 
 NaN 
 -0.369283 
 1.000000 
 0.177833 
 0.194921 
 
 
 StO2_1 
 -0.843294 
 0.206943 
 -0.176011 
 NaN 
 -0.379369 
 0.177833 
 1.000000 
 -0.127283 
 
 
 StO2_2 
 0.176806 
 -0.887300 
 -0.726473 
 NaN 
 -0.203

In [93]:
print('All Subjects 1 Hour Before')
display(dfb.corr().style.highlight_max(axis=0))

print('All Subjects 1 Hour After')
display(dfa.corr().style.highlight_max(axis=0))

All Subjects 1 Hour Before


,HR,RR,SpO2,MAP,StO2_1,StO2_2,FTOE_1,FTOE_2
HR,1,0.337959,-0.293449,0.0686795,-0.350004,-0.649776,0.238114,0.55234
RR,0.337959,1,-0.107343,-0.0443829,-0.218707,0.0173723,0.194007,0.0964979
SpO2,-0.293449,-0.107343,1,-0.140892,0.294508,0.267308,0.22084,0.162695
MAP,0.0686795,-0.0443829,-0.140892,1,0.0168587,-0.336472,-0.100161,0.268642
StO2_1,-0.350004,-0.218707,0.294508,0.0168587,1,0.400254,-0.865801,-0.191975
StO2_2,-0.649776,0.0173723,0.267308,-0.336472,0.400254,1,-0.197369,-0.905793
FTOE_1,0.238114,0.194007,0.22084,-0.100161,-0.865801,-0.197369,1,0.307379
FTOE_2,0.55234,0.0964979,0.162695,0.268642,-0.191975,-0.905793,0.307379,1


All Subjects 1 Hour After


,HR,RR,SpO2,MAP,StO2_1,StO2_2,FTOE_1,FTOE_2
HR,1,0.128486,0.0140661,0.42578,-0.367634,0.340272,0.437924,-0.352486
RR,0.128486,1,-0.111388,0.0108593,0.0471967,-0.0107289,-0.131769,-0.08338
SpO2,0.0140661,-0.111388,1,-0.455961,0.247085,0.22744,0.206343,0.195993
MAP,0.42578,0.0108593,-0.455961,1,-0.657749,-0.138463,0.351094,-0.227041
StO2_1,-0.367634,0.0471967,0.247085,-0.657749,1,0.336217,-0.894414,-0.29352
StO2_2,0.340272,-0.0107289,0.22744,-0.138463,0.336217,1,-0.291237,-0.90875
FTOE_1,0.437924,-0.131769,0.206343,0.351094,-0.894414,-0.291237,1,0.384845
FTOE_2,-0.352486,-0.08338,0.195993,-0.227041,-0.29352,-0.90875,0.384845,1


In [26]:
'''
for i in varlst[1:-1]:
    dfb.boxplot(by='Subject', column=i)
    plt.title(i + ' 1 Hr Before Procedure')
    plt.subplots_adjust(top=0.9,
    bottom=0.15,
    left=0.1,
    right=0.9,
    hspace=0.2,
    wspace=0.2)
    plt.savefig('figs/' + i + '_1_Hr_Before_Procedure.png')

for i in varlst[1:-1]:
    dfa.boxplot(by='Subject', column=i)
    plt.title(i + ' 1 Hr After Procedure')
    plt.subplots_adjust(top=0.9,
    bottom=0.15,
    left=0.1,
    right=0.9,
    hspace=0.2,
    wspace=0.2)
    plt.savefig('figs/' +  i + '_1_Hr_After_Procedure.png')
'''

In [12]:
# data.to_csv('clean/GTP_raw.csv', index=False)